In [1]:
!pip install openai python-dotenv --quiet


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
import os
load_dotenv()

from openai import OpenAI
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
load_dotenv()  
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
from pathlib import Path

def build_prompt(claim: str) -> str:
    prompt_path = Path("../src/rebuttal/prompts.txt")  # ← 从 notebooks/ 向上一级，再进 src/rebuttal/
    
    if not prompt_path.exists():
        raise FileNotFoundError(f"prompts.txt not found at {prompt_path}")

    with open(prompt_path, 'r') as file:
        template = file.read()

    return template.replace('{claim}', claim)

In [5]:
def generate_rebuttal(claim: str) -> str:
    """
    Given a claim, call OpenAI ChatCompletion API and return the response.
    Includes basic fallback if response is incomplete.
    """
    prompt = build_prompt(claim)

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",  # or "gpt-4"
            messages=[
                {"role": "system", "content": "You are a helpful medical fact-checking assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5,
            max_tokens=300
        )

        reply = response.choices[0].message.content.strip()

        if "Verdict" not in reply or "Explanation" not in reply:
            return "⚠️ Could not generate a reliable rebuttal."
        
        return reply

    except Exception as e:
        return f"⚠️ API error: {str(e)}"

In [6]:
claims = [
    "Vaccines contain microchips for tracking people.",
    "COVID vaccines cause infertility in women.",
    "mRNA vaccines change your DNA.",
    "You don't need vaccines if you take enough vitamins.",
    "Vaccinated people shed spike proteins and harm others.",
    "COVID-19 was planned to sell vaccines.",
    "Natural immunity is always better than vaccine immunity.",
    "COVID vaccines were developed too quickly to be safe.",
    "5G networks caused the coronavirus pandemic.",
    "The vaccine has a magnetic effect on your body."
]

In [7]:
def generate_and_save_rebuttals(claims, output_path='rebuttal_outputs.txt'):
    """
    Given a list of claims, generate rebuttals and save them to a text file.
    """
    all_outputs = []
    
    for i, claim in enumerate(claims, 1):
        print(f"📝 Generating for claim {i}/{len(claims)}...")
        response = generate_rebuttal(claim)
        formatted = f"Claim {i}: {claim}\nRebuttal:\n{response}\n{'-'*60}\n"
        all_outputs.append(formatted)
    
    with open(output_path, 'w') as f:
        f.writelines(all_outputs)
    
    print(f"\n✅ Finished. All rebuttals saved to {output_path}")

In [8]:
generate_and_save_rebuttals(claims)

📝 Generating for claim 1/10...
📝 Generating for claim 2/10...
📝 Generating for claim 3/10...
📝 Generating for claim 4/10...
📝 Generating for claim 5/10...
📝 Generating for claim 6/10...
📝 Generating for claim 7/10...
📝 Generating for claim 8/10...
📝 Generating for claim 9/10...
📝 Generating for claim 10/10...

✅ Finished. All rebuttals saved to rebuttal_outputs.txt
